In [1]:
import pandas
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

2022-05-30 10:06:58.386487: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# data path
data_path = '/data/linghui/ml-1m/'
# data_path = '/home/nesa320/huizhong/movielens/data-ml/'
# file : ratings.dat  movies.dat  users.dat

user_file = data_path + 'users.dat'
rating_file = data_path + 'ratings.dat'
movie_file = data_path + 'movies.dat'

In [24]:
# load user data
user_data = pd.read_csv(user_file, sep='::', header=None)
user_data.columns = ['user_id',  'gender', 'age','occupation', 'zipcode']
lbe_feats = ['user_id',  'gender', 'age','occupation', 'zipcode']
for feat in lbe_feats:
    lbe = LabelEncoder()
    user_data[feat] = lbe.fit_transform(user_data[feat])
user_data

/opt/conda/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,user_id,gender,age,occupation,zipcode
0,0,0,0,10,1588
1,1,1,6,16,2248
2,2,1,2,15,1863
3,3,1,4,7,140
4,4,1,2,20,1938
...,...,...,...,...,...
6035,6035,0,2,15,1152
6036,6036,0,4,1,2367
6037,6037,0,6,1,626
6038,6038,0,4,0,13


In [25]:
#load rating data

rating_data = pd.read_csv(rating_file, sep='::', header=None,error_bad_lines=False,engine='python')
rating_data.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
# rating_data = rating_data.dropna()
# # rating_data['movie_id'] = rating_data['movie_id'].apply(pd.to_numeric,errors='coerce')

# rating_data['movie_id'] = rating_data['movie_id'].astype('int') 
# rating_data['timestamp'] = rating_data['timestamp'].round(0).astype('int') 
lbe_feats = ['user_id', 'movie_id']
for feat in lbe_feats:
    lbe = LabelEncoder()
    rating_data[feat] = lbe.fit_transform(rating_data[feat])

rating_data

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,user_id,movie_id,rating,timestamp
0,0,1104,5,978300760
1,0,639,3,978302109
2,0,853,3,978301968
3,0,3177,4,978300275
4,0,2162,5,978824291
...,...,...,...,...
1000204,6039,1019,1,956716541
1000205,6039,1022,5,956704887
1000206,6039,548,5,956704746
1000207,6039,1024,4,956715648


In [5]:
rating_data['movie_id'][0]

1193

In [26]:
# load movie data

movie_data = pd.read_csv(movie_file, sep='::', header=None, error_bad_lines=False,engine='python', encoding="ISO-8859-1")
movie_data.columns = ['movie_id', 'title', 'genres']
lbe_feats = ['movie_id']
for feat in lbe_feats:
    lbe = LabelEncoder()
    movie_data[feat] = lbe.fit_transform(movie_data[feat])
movie_data

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,movie_id,title,genres
0,0,Toy Story (1995),Animation|Children's|Comedy
1,1,Jumanji (1995),Adventure|Children's|Fantasy
2,2,Grumpier Old Men (1995),Comedy|Romance
3,3,Waiting to Exhale (1995),Comedy|Drama
4,4,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3878,Meet the Parents (2000),Comedy
3879,3879,Requiem for a Dream (2000),Drama
3880,3880,Tigerland (2000),Drama
3881,3881,Two Family House (2000),Drama


In [29]:
def create_ml_1m_data(data):
    data = data.sort_values(['user_id','timestamp'])
    num = len(data.user_id.value_counts())
    test_pd = pd.DataFrame(columns=data.columns)
    for i in range(num):
        user_select_group = data.loc[data['user_id'] == i]
        user_pd = user_select_group.iloc[-1]
        test_pd = test_pd.append(user_pd)
    drop_index = test_pd.index.to_list()
    train_pd = data[~data.index.isin(drop_index)]
    return train_pd,test_pd

def get_user_hist(data):
    num = len(data.user_id.value_counts()) + 1
    hist_list = []
    for i in range(1,num):
        user_select_group = data.loc[data['user_id'] == i]
        user_hist = user_select_group['movie_id'].to_list()
        hist_list.append(user_hist)
    return 
  

In [30]:
train_set , test_set = create_ml_1m_data(rating_data)

In [31]:
train_set

,user_id,movie_id,rating,timestamp
31,0,2969,4,978300019
22,0,1178,5,978300055
27,0,1574,4,978300055
37,0,957,5,978300055
24,0,2147,3,978300103
...,...,...,...,...
999923,6039,225,5,997454398
1000019,6039,2709,4,997454429
999988,6039,1741,4,997454464
1000172,6039,1618,3,997454464


In [32]:
# add negative sample 

def neg_sample(u_data, neg_rate_train=1,neg_rate_test = 49):
    item_ids = u_data['movie_id'].unique()
    print('start neg sample')
    neg_data_train = []
    neg_data_test = []
    for user_id, hist in tqdm(u_data.groupby('user_id')):
        rated_movie_list = hist['movie_id'].tolist()
        candidate_set = list(set(item_ids) - set(rated_movie_list))
        neg_list_train = np.random.choice(candidate_set, size=(len(rated_movie_list)-1) * neg_rate_train, replace=True)
        neg_list_test = np.random.choice(candidate_set, size=neg_rate_test, replace=True)
        for id in neg_list_train:
            neg_data_train.append([user_id, id, -1, 0])
        for id in neg_list_test:
            neg_data_test.append([user_id, id, -1, 0])
    u_data_neg_train = pd.DataFrame(neg_data_train)
    u_data_neg_test = pd.DataFrame(neg_data_test)
    u_data_neg_train.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
    u_data_neg_test.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
    return u_data_neg_train,u_data_neg_test

u_data_neg_train,u_data_neg_test = neg_sample(rating_data, neg_rate_train=1,neg_rate_test=49)
train_set = pd.concat([train_set, u_data_neg_train])
test_set = pd.concat([test_set, u_data_neg_test])


start neg sample


100%|██████████| 6040/6040 [00:11<00:00, 538.57it/s]


In [19]:
test_set.loc[test_set['user_id'] == 1]

,user_id,movie_id,rating,timestamp
25,1,48,5,978824351
0,1,716,-1,0
1,1,3089,-1,0
2,1,1068,-1,0
3,1,1237,-1,0
4,1,2821,-1,0
5,1,3259,-1,0
6,1,2541,-1,0
7,1,2724,-1,0
8,1,1527,-1,0


In [33]:
#merge three data on movie_id , user_id

train_set = pd.merge(train_set, movie_data, on="movie_id", how='left')
train_set = pd.merge(train_set, user_data, on="user_id", how='left')

test_set = pd.merge(test_set, movie_data, on="movie_id", how='left')
test_set = pd.merge(test_set, user_data, on="user_id", how='left')

In [39]:
select_score = 0
train_set['rating'] = [1 if int(x) >= select_score else 0 for x in train_set['rating']]
test_set['rating'] = [1 if int(x) >= select_score else 0 for x in test_set['rating']]

In [40]:
train_set

,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
0,0,2969,1,978300019,"Face in the Crowd, A (1957)",Drama,0,0,10,1588
1,0,1178,1,978300055,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War,0,0,10,1588
2,0,1574,1,978300055,"Peacemaker, The (1997)",Action|Thriller|War,0,0,10,1588
3,0,957,1,978300055,"African Queen, The (1951)",Action|Adventure|Romance|War,0,0,10,1588
4,0,2147,1,978300103,"Skin Game, The (1931)",Drama,0,0,10,1588
...,...,...,...,...,...,...,...,...,...,...
1988333,6039,1074,0,0,Platoon (1986),Drama|War,1,2,6,466
1988334,6039,627,0,0,Land and Freedom (Tierra y libertad) (1995),War,1,2,6,466
1988335,6039,2882,0,0,"Fistful of Dollars, A (1964)",Action|Western,1,2,6,466
1988336,6039,3356,0,0,Mo' Better Blues (1990),Drama,1,2,6,466


In [41]:
saved_data_path = '/data/linghui/ml-1m/processed_data/'
train_set.to_csv(saved_data_path + 'train_data_liuyi.csv')
test_set.to_csv(saved_data_path + 'test_data_liuyi.csv')

In [55]:
import math
from math import log
log(3,2)


1.5849625007211563

In [ ]:
def cal_ndcg(data):
    user_id = data['user_id'].unique()
    ndcg_list = []
    for i in user_id:
        user_select_group = data.loc[data['user_id'] == i]
        user_select_group = user_select_group.reset_index(drop = True)
        rec_pos = user_select_group.loc[user_select_group['rating'] == 1]
        if len(rec_pos) == 0:
            ndcg = 0
        else:
            pos_index = rec_pos.index[0] + 1  
            pre_score = rec_pos.iloc[0].loc['predict']
            ndcg = (pow(2,pre_score)-1) / (log(pos_index,2)) 
        ndcg_list.append(arhr)
    return np.mean(ndcg_list)